## Analysis of a Stabilization graph
Based on Arie Landau *et al.*, *J. Phys. Chem.* A 2016, 120, 3098−3108

Analytic continuation of a single root using standard Pade approximants and input data from a plateau. 

In [1]:
import numpy as np
#from scipy.optimize import minimize_scalar
#from scipy.optimize import root
#from pandas import Series, DataFrame
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')
%matplotlib qt5
Angs2Bohr=1.8897259886
au2eV=27.211386027
au2cm=219474.63068

In [2]:
#
# put pade.py into the current directory or put its location into the system path
#
import sys
sys.path.append('../../Python_libs')
import pade
import stabtools as st

Landau outlines a ten-step protocol:

### Step 1: Get a real stabilization graph

The scaling parameter $z$ may be a box length $L$  or a scaling factor $\alpha$ of a set of Gaussians.

In [11]:
#df = pd.read_csv("DVR_stab_plot.csv")
#df = pd.read_csv("GTO_unc_stab_plot.csv")
#df = pd.read_csv("GTO_DZ_stab_plot.csv")
df = pd.read_csv("GTO_TZ_stab_plot.csv")
df.head()

,L,z,E1,E2,E3,E4,E5,E6,E7
0,1.825742,0.300000,-7.170448,0.087908,0.311252,0.744851,1.887052,3.208072,6.198476
1,1.795871,0.310063,-7.170447,0.091174,0.323694,0.776332,1.973841,3.228715,6.323012
2,1.767420,0.320126,-7.170447,0.094467,0.336242,0.808294,2.059987,3.250052,6.445424
3,1.740279,0.330189,-7.170447,0.097789,0.348895,0.840741,2.145085,3.272481,6.565790
4,1.714352,0.340252,-7.170447,0.101138,0.361654,0.873671,2.228683,3.296428,6.684194


In [12]:
z_col=df.columns[1]
zs=df[z_col].values
zmin, zmax = zs[0], zs[-1]
if zmax < zmin:
    zmin, zmax = zmax, zmin
zs=(zs-zmin)/(zmax-zmin)

E_labels=df.columns[2:]
if True:
    plt.cla()
    for E in E_labels:
        plt.plot(zs, df[E].values, '.')
    plt.ylim(0,8)
    plt.show()

### Step 2: Select the low curvature region of a plateu 

Set the following parameters after inspecting the stabilization plot:
* lroot: root with a stabilization plateau
* if the autoselect doesn't work, narrow search by setting zmin, zmax

In [18]:
lroot=4
zmin, zmax = -1, -1
E_0=df[E_labels[lroot]].values
d1s, d2s = st.der_and_curvature(zs,E_0)
(jc, j1, j2), (zc, z1, z2) = st.plateau(zs,E_0,srch_range=(zmin,zmax))
if jc > 0:
    print('Plateau center at z=%f' % (zc))
if j1 > 0 and j2 > 0:
    print('Crossings at z=%f and z=%f' % (z1, z2))
    if j1 > j2:
        j1, j2 = j2, j1
    d2dn, d2up = d2s[j1], d2s[j2]
    ipdn=j1+np.argmin(np.abs(d2s[j1:j2]-0.5*d2dn))
    ipup=j1+np.argmin(np.abs(d2s[j1:j2]-0.5*d2up))
    alps=zs[ipdn:ipup+1]
    Es=E_0[ipdn:ipup+1]
    npts = len(Es)
    print('N = %d,  max n: %d' % (npts, (npts-1)/2))
    plt.cla()
    plt.plot(zs, E_0, '-')
    plt.plot(alps, Es, 'o')
    plt.ylabel('root '+str(lroot))
    plt.show()
else:
    print('No plateau with adjacent crossings found.')
    print(jc, j1, j2)
    plt.cla()
    plt.plot(zs,d1s,label="1st der")
    plt.plot(zs,d2s,label="2nd der")
    plt.legend()
    plt.ylabel('derivative and curvature')
    plt.show()

Plateau center at z=0.214431
Crossings at z=0.069182 and z=0.345912
N = 28,  max n: 13


### Step 3 and 4: Fit to [$n$,$n$]-Pade approximant and show the result

In [19]:
n=4
P, Q = pade.pade_via_lstsq(n, n, alps, Es)
print('E = P/Q')
print('P :\n ', P)
print('Q :\n ', Q)

#
# compare the input energies and the fitted curves
#
npts=200
aplot=np.linspace(zs[0],zs[-1],npts)
Eplot=P(aplot)/Q(aplot)
#for i in range(npts):
#    E1plot[i]=pade.E_lower(Lplots[i], A, B, C)
#    E2plot[i]=pade.E_upper(Lplots[i], A, B, C)
plt.cla()
plt.plot(zs, E_0, '-.')
plt.plot(alps, Es, 'o')
plt.plot(aplot,Eplot,'-')
plt.ylim(0,6)
plt.show()

E = P/Q
P :
        4        3        2
8474 x - 7409 x + 1962 x - 157.4 x + 3.814
Q :
        4        3         2
2689 x - 2302 x + 593.9 x - 45.2 x + 1


### Steps $5-8$

Instead of computing $\theta$ and $\alpha$ trajectories,
* Evaluate the derivative of the Pade approximant in the complex plane.
* Plot its log-abs on a colorscale.
* Read the approximate stable point off the graph.
* Find the precise stable point with Newton-Raphson.

In [20]:
npts=81
rp=1.2 # plus/minus x%
ip=2*zc
Lrs = np.linspace(alps[0]*(1-rp), alps[-1]*(1+rp), npts)
Lis = np.linspace(-ip, 0, npts)
der = np.zeros([npts,npts])
for r in range(npts):
    for i in range(npts):
        L=Lrs[r]+ 1j*Lis[i]
        der[i,r] = np.log10(abs(pade.dEds(L, P, Q)))

plt.cla() 
plt.contourf(Lrs, Lis, der, 10, cmap=plt.cm.jet_r)
plt.colorbar()
plt.show()

In [21]:
#   read-off from graph, derivative and Newton step should be small

#L_guess=0.13+1j*0.33  # DVR
#L_guess=0.40+1j*0.22  # DVR
#L_guess=0.26+1j*0.34  # GTO3
L_guess=0.2-1j*0.2  


der = pade.dEds(L_guess, P, Q)
print("dE/ds =", der)
corr = pade.EpoEpp(L_guess, P, Q)
print("E'/E'' =", corr)

dE/ds = (-0.021582725771307865-0.06777080136364597j)
E'/E'' = (0.017796884946345556-0.01026448932237077j)


In [22]:
# Newton Raphson

max_step = 20
Lstar = L_guess
Ecurr = P(Lstar)/Q(Lstar)
tol=1e-7
for i in range(max_step):
    delta_L = pade.EpoEpp(Lstar, P, Q)
    Lstar = Lstar - delta_L
    Enew = P(Lstar)/Q(Lstar)
    delta_E = Enew - Ecurr
    Ecurr = Enew
    # print table with L E
    print("%3d  (%.7f, %.7f)   (%.7f, %.7f)" % 
          (i, Lstar.real, Lstar.imag, Ecurr.real, Ecurr.imag))
    # check convergence
    if abs(delta_L) < tol and delta_E < tol:
        break
print("\nFinal results:")
print("  L_star = (%.6f, %.6f)" % (Lstar.real, Lstar.imag))
print("  Er = %.6f,  Ei = %.6f" % (Ecurr.real, Ecurr.imag))
# table = (n,Lstar.real, Lstar.imag,Ecurr.real, 2*Ecurr.imag)
# print('| %1d | (%.6f, %.6f) | %.6f | %.6f |' % table)

  0  (0.1822031, -0.1897355)   (3.1863010, -0.1451942)
  1  (0.1850608, -0.1885619)   (3.1862825, -0.1451892)
  2  (0.1850064, -0.1885090)   (3.1862825, -0.1451892)
  3  (0.1850064, -0.1885090)   (3.1862825, -0.1451892)

Final results:
  L_star = (0.185006, -0.188509)
  Er = 3.186282,  Ei = -0.145189


To do **steps 9 and 10**: 
* Go back and repeat with different $n$.
* Use a different percentage than the default.